In [178]:
import os
import re
import tarfile

import numpy as np
import pandas as pd

from tqdm import tqdm

pd.set_option('display.max_colwidth', -1)

In [118]:
df = pd.read_csv('/home/scimpian/grepner_pubmed_2016.tsv', sep='\t', header=None)

In [119]:
df.head()

,0,1,2,3,4,5,6,7
0,/scratch0/psteneto/medline2016/medline16n0001/16691752.txt.ss.tok.grepner,1,0,90,17,22,Water,DB09145
1,/scratch0/psteneto/medline2016/medline16n0001/16691813.txt.ss.tok.grepner,1,0,67,24,28,Iron,DB01592
2,/scratch0/psteneto/medline2016/medline16n0001/16991250.txt.ss.tok.grepner,1,0,147,126,130,Urea,DB03904
3,/scratch0/psteneto/medline2016/medline16n0001/16991252.txt.ss.tok.grepner,1,0,61,43,50,Atropin,DB00572
4,/scratch0/psteneto/medline2016/medline16n0001/16991253.txt.ss.tok.grepner,1,0,19,12,16,Prof,Q96P53


In [120]:
df['base'] = df[0].apply(lambda path: path.split('/')[4])

In [121]:
df['file'] = df[0].apply(lambda path: path.split('/')[5][:-8])

In [122]:
df = df.drop([0, 1, 2, 3], axis=1)

In [123]:
df.head()

,4,5,6,7,base,file
0,17,22,Water,DB09145,medline16n0001,16691752.txt.ss.tok
1,24,28,Iron,DB01592,medline16n0001,16691813.txt.ss.tok
2,126,130,Urea,DB03904,medline16n0001,16991250.txt.ss.tok
3,43,50,Atropin,DB00572,medline16n0001,16991252.txt.ss.tok
4,12,16,Prof,Q96P53,medline16n0001,16991253.txt.ss.tok


In [147]:
gr = df.groupby(['base', 'file']).agg({4: list, 5: list, 6: list, 7: list})

In [148]:
gr.head()

4      5          6          7
base           file                                                   
medline16n0001 16691752.txt.ss.tok  [17]   [22]   [Water]    [DB09145]
               16691813.txt.ss.tok  [24]   [28]   [Iron]     [DB01592]
               16991250.txt.ss.tok  [126]  [130]  [Urea]     [DB03904]
               16991252.txt.ss.tok  [43]   [50]   [Atropin]  [DB00572]
               16991253.txt.ss.tok  [12]   [16]   [Prof]     [Q96P53]

In [152]:
gr = gr.reset_index()

In [153]:
gr.head()

,base,file,4,5,6,7
0,medline16n0001,16691752.txt.ss.tok,[17],[22],[Water],[DB09145]
1,medline16n0001,16691813.txt.ss.tok,[24],[28],[Iron],[DB01592]
2,medline16n0001,16991250.txt.ss.tok,[126],[130],[Urea],[DB03904]
3,medline16n0001,16991252.txt.ss.tok,[43],[50],[Atropin],[DB00572]
4,medline16n0001,16991253.txt.ss.tok,[12],[16],[Prof],[Q96P53]


In [156]:
len(gr.loc[gr['file'] == 'a'])

0

In [182]:
regex = re.compile('^(?:.)+\.tar\.gz$')

In [188]:
replaced = open('replaced.txt', 'w')

In [ ]:
for path in tqdm(os.listdir('/home/psteneto/mr_corpora/psteneto/pubmed_2016_xml_txt_ss_tok/tok/')):
    if regex.match(path):
        tar = tarfile.open('/home/psteneto/mr_corpora/psteneto/pubmed_2016_xml_txt_ss_tok/tok/{}'.format(path))
        base = path.split('.tar.gz')[0]
        subgr = gr.loc[gr['base'] == base]
        for member in tar.getmembers():
            file = member.name.split('/')[1]
            repl = subgr.loc[subgr['file'] == file]
            if len(repl) == 1:
                f = tar.extractfile(member.name)
                content = f.read()
                new_content = ''
                str_start = 0
                ok = True
                for start, end, elem, elem_id in zip(repl[4].iloc[0], repl[5].iloc[0], repl[6].iloc[0], repl[7].iloc[0]):
                    if elem != content[start:end].decode('utf-8'):
                        print(elem, content[start:end])
                        ok = False
                        break
                    new_content += content[str_start:start].decode('utf-8') + elem_id
                    str_start = end
                if ok:
                    print(new_content.encode('utf-8'), file=replaced)


  3%|▎         | 49/1624 [10:12<5:28:18, 12.51s/it]

nan b'N/A'



  5%|▍         | 78/1624 [17:28<5:46:12, 13.44s/it]

nan b'N/A'



  6%|▌         | 92/1624 [20:56<5:48:49, 13.66s/it]

...


  9%|▉         | 146/1624 [34:15<5:46:45, 14.08s/it]

nan b'N/A'
nan b'N/A'
nan b'N/A'
nan b'N/A'
nan b'N/A'
nan b'N/A'
nan b'N/A'



 11%|█         | 178/1624 [41:45<5:39:12, 14.07s/it]

nan b'N/A'
nan b'N/A'



 12%|█▏        | 195/1624 [46:07<5:37:59, 14.19s/it]